In [ ]:
# make battery usage plot
# load receiver/log.txt
from pathlib import Path

import altair as alt
import pandas as pd

In [15]:
log_file = Path("receiver/log.txt")
lines = log_file.read_text().splitlines()

raw_data = []
for line in lines[5:]:
    # parse csv line
    if " [INFO] " not in line:
        continue
    timestamp, data = line.split(" [INFO] ", 1)
    try:
        voltage, ax, ay, az = map(float, data.split(","))
        raw_data.append(
            {"timestamp": timestamp, "voltage": voltage, "ax": ax, "ay": ay, "az": az}
        )
    except ValueError:
        pass
    # voltage, ax, ay, az = map(float, data.split(','))
df = pd.DataFrame(raw_data)
df["timestamp"] = pd.to_datetime(df["timestamp"])

In [16]:
df

,timestamp,voltage,ax,ay,az
0,2025-11-01 03:56:09,2.88,6.82,-0.66,-7.02
1,2025-11-01 03:56:10,2.87,6.82,-0.65,-7.03
2,2025-11-01 03:56:11,2.88,6.82,-0.66,-7.03
3,2025-11-01 03:56:12,2.87,6.82,-0.65,-7.03
4,2025-11-01 03:56:13,2.88,6.82,-0.65,-7.01
...,...,...,...,...,...
5363,2025-11-01 05:26:12,2.33,6.93,-0.65,-6.91
5364,2025-11-01 05:26:13,2.31,6.93,-0.66,-6.90
5365,2025-11-01 05:26:14,2.33,6.93,-0.65,-6.91
5366,2025-11-01 05:26:15,2.30,6.94,-0.65,-6.91


In [18]:
# downsample volatage to 1 reading per minute
df = df.resample("1T", on="timestamp").mean().reset_index()

C:\Users\lcmjl\AppData\Local\Temp\ipykernel_9212\4119661386.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample("1T", on="timestamp").mean().reset_index()


In [19]:
# plot voltage over time
alt.Chart(df).mark_line().encode(
    x="timestamp:T",
    y="voltage:Q",
    tooltip=["timestamp:T", "voltage:Q"],
).properties(
    title="Battery Voltage Over Time",
).interactive()

alt.Chart(...)

In [ ]:
# calculate scalar acceleration
df["acceleration"] = (df["ax"] ** 2 + df["ay"] ** 2 + df["az"] ** 2) ** 0.5
alt.Chart(df).mark_line().encode(
    x="timestamp:T",
    y="acceleration:Q",
    tooltip=["timestamp:T", "acceleration:Q"],
).properties(
    title="Scalar Acceleration Over Time",
).interactive()

alt.Chart(...)

### Fall back Kalman filter

If power budget is not enough, consider upgrading to Kalman filter

In [ ]:
import numpy as np


class LinearAccelKF:
    """
    Estimate gravity and linear acceleration from accelerometer-only data
    using a 6-state Kalman filter.
    State: [g_bx, g_by, g_bz, a_lx, a_ly, a_lz]
    """

    def __init__(self, fs=100.0, phi=0.98, qg=1e-3, qa=1e-1, r=0.05):
        self.F = np.block(
            [
                [np.eye(3), np.zeros((3, 3))],
                [np.zeros((3, 3)), phi * np.eye(3)],
            ]
        )
        self.H = np.hstack([np.eye(3), np.eye(3)])
        # Process / measurement noise
        self.Q = np.diag([qg] * 3 + [qa] * 3)
        self.R = np.eye(3) * r
        self.x = np.zeros(6)
        self.P = np.eye(6) * 1.0

    def initialize_gravity(self, accel_samples):
        """Use average of stationary samples for initial gravity estimate."""
        g0 = np.mean(accel_samples, axis=0)
        self.x[:3] = g0
        self.P = np.eye(6) * 0.1

    def update(self, z):
        """Perform one EKF step given accel measurement z (3-vector)."""
        # predict
        x_pred = self.F @ self.x
        P_pred = self.F @ self.P @ self.F.T + self.Q

        # measurement update
        y = z - self.H @ x_pred
        S = self.H @ P_pred @ self.H.T + self.R
        K = P_pred @ self.H.T @ np.linalg.inv(S)
        self.x = x_pred + K @ y
        self.P = (np.eye(6) - K @ self.H) @ P_pred

        g_est = self.x[:3]
        a_lin_est = self.x[3:]
        return g_est, a_lin_est


In [ ]:
fs = 100.0
ekf = LinearAccelKF(fs=fs)
# simulate 1 s stationary data for initialization
init_data = np.tile(np.array([0, 0, 9.81]), (100, 1)) + np.random.randn(100, 3) * 0.05
ekf.initialize_gravity(init_data)

# simulate 2 s of small movement
accel_data = np.tile(np.array([0, 0, 9.81]), (200, 1))
accel_data[50:100, 0] += 1.0  # brief x acceleration

for a in accel_data:
    g, a_lin = ekf.update(a)
    print(f"g: {g}, a_linear: {a_lin}")
    # print or log a_lin if desired

g: [-1.13855377e-04  3.27376411e-03  9.80682269e+00], a_linear: [ 9.07178021e-05 -2.60847308e-03  2.53162047e-03]
g: [-1.11693912e-04  3.21161397e-03  9.80688301e+00], a_linear: [ 0.00010522 -0.00302545  0.00293632]
g: [-1.11387628e-04  3.20280718e-03  9.80689156e+00], a_linear: [ 0.00010909 -0.00313677  0.00304435]
g: [-1.11286933e-04  3.19991185e-03  9.80689437e+00], a_linear: [ 0.00011008 -0.00316534  0.00307208]
g: [-1.11221886e-04  3.19804150e-03  9.80689618e+00], a_linear: [ 0.00011031 -0.00317176  0.00307831]
g: [-1.11164716e-04  3.19639764e-03  9.80689778e+00], a_linear: [ 0.00011033 -0.00317231  0.00307884]
g: [-1.11109257e-04  3.19480298e-03  9.80689932e+00], a_linear: [ 0.00011029 -0.0031713   0.00307786]
g: [-1.11053961e-04  3.19321302e-03  9.80690087e+00], a_linear: [ 0.00011024 -0.00316987  0.00307647]
g: [-1.10998425e-04  3.19161615e-03  9.80690242e+00], a_linear: [ 0.00011019 -0.00316832  0.00307497]
g: [-1.10942543e-04  3.19000935e-03  9.80690398e+00], a_linear: [ 0.00